# Current Work

In [45]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import os
import json

# notebook settings
pd.set_option('display.max_columns', None)

In [46]:
path = "./unsynced-data/crop-yield-data/"
crop_files = os.listdir(path)
crop_files = [f for f in crop_files if f.endswith('.csv')]
grid = gpd.read_file("./synced-data/empty-grid.geojson")


In [47]:
def reduce_simulations(df, string, grid_df):
    working_df = df[["Exp.", "Yield", "Biomass", "Duration", "lat", "long"]].groupby("Exp.").mean().reset_index()
    # clean up and turn df into geodataframe
    working_df.columns = map(str.lower, working_df.columns)
    working_df = working_df.rename(columns={"yield": "yield_" + string, "biomass": "biomass_" + string, "duration": "duration_" + string})
    working_df = gpd.GeoDataFrame(working_df, crs="EPSG:4326", geometry=gpd.points_from_xy(working_df['long'], working_df['lat']))
    working_df = working_df.drop(columns=["lat", "long", "exp."])
    # add working_df to grid_df where geometries intersect on geometry
    grid_df = gpd.sjoin(grid_df, working_df, how="left", op="intersects")
    grid_df = grid_df.drop(columns='index_right')
    grid_df.columns = map(str.lower, grid_df.columns)
    return grid_df

In [48]:
def reduce_models(df):
    # create a list of column names
    column_names = df.columns.tolist()
    column_names.remove("geometry")
    models = ["_ipsl", "_mpi", "_gfdl", "_mri"]
    
    for m in models:
        # remove  m from the column names
        column_names = [c.replace(m, "") for c in column_names]
    # remove duplicates in column_names list by converting to set and back to list
    column_names = list(set(column_names))
    # average all columns that match a column name
    for c in column_names:
        df[c] = df.filter(regex=c).mean(axis=1)
    # drop all columns that include a string from models
    df = df.drop(columns=[c for c in df.columns if any(m in c for m in models)])
    df = df.round(3)
    return df


In [49]:
# It feels like there should be a better way to remove/not write NAN's to a geojson file to save space

def geojson_no_nans(df, path):
    # First write a geopandas dataframe to a geojson file
    df.to_file(path, driver='GeoJSON')

    # Load the GeoJSON data from a file
    with open(path, 'r') as geojson_file:
        geojson_data = json.load(geojson_file)

    def remove_null_properties(feature):
        non_null_properties = {key: value for key, value in feature['properties'].items() if value is not None}
        feature['properties'] = non_null_properties

    # Loop through features and remove null properties
    for feature in geojson_data['features']:
        remove_null_properties(feature)

    # Save the modified GeoJSON to a new file
    with open(path, 'w') as output_file:
        json.dump(geojson_data, output_file, indent=2)

In [50]:
# a dictionary of column quanitiles is useful for styling the map in the frontend
def generate_quantiles(df):
    col_list = df.columns.tolist()
    col_list.remove("geometry")
    q_list = ["min", 0.01, 0.02, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.98, 0.99, "max"]
    working_dict = {}
    for col in col_list:
        quantile_dict = {}
        for quantile in q_list:
            if quantile == "min":
                quantile_value = df[col].min()
                quantile_dict[quantile] = quantile_value

            elif quantile == "max":
                quantile_value = df[col].max()
                quantile_dict[quantile] = quantile_value
            else:
                quantile_value = df[col].quantile(quantile)
                quantile_dict[f"quantile_{int(quantile * 100)}"] = quantile_value  
        working_dict[col] = quantile_dict
    return working_dict

In [51]:
# i want to copy the generate_quantiles function, but want to make it the ratio of the ssp126/historical columns and the ssp360/historical columns
def generate_ratio_quantiles(df):
    # col_list = all columns that include ssp126 or ssp360
    col_list = [c for c in df.columns if "ssp126" in c or "ssp370" in c]
    col_list = [c for c in col_list if "yield" in c]
    # yield_sorghum_future_ssp370
    q_list = ["min", 0.01, 0.02, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.98, 0.99, "max"]
    working_dict = {}
    for col in col_list:
        quantile_dict = {}
        crop = col.split("_")[1]
        # divide the column by the "yield_"+crop+"_future_ssp370"column
        df["working_col"] = df[col] / df["yield_"+crop+"_historical"]
        for quantile in q_list:
            if quantile == "min":
                quantile_value = df["working_col"].min()
                quantile_dict[quantile] = quantile_value
                if quantile_value == np.inf:
                    quantile_value = "none"
            elif quantile == "max":
                quantile_value = df["working_col"].max()
                if quantile_value == np.inf:
                    quantile_value = "none"
                quantile_dict[quantile] = quantile_value
            else:
                quantile_value = df["working_col"].quantile(quantile)
                quantile_dict[f"quantile_{int(quantile * 100)}"] = quantile_value  
                if quantile_value == "NaN":
                    quantile_value = "none"
        working_dict[col] = quantile_dict
    return working_dict

In [52]:
for c in crop_files:
    print (c)
    file_df = pd.read_csv(path + c)
    column_string = c.split("-")[0]
    grid = reduce_simulations(file_df, column_string, grid)
grid = grid.round(3)


okra_historical_SSP370_MRI-ESM2-0_Africa.csv
taro_future_SSP370_MPI-ESM1-2-HR_Africa.csv
fonio_future_SSP126_MRI-ESM2-0_Africa.csv
maize_historical_SSP370_MPI-ESM1-2-HR_Africa.csv


/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/IPython/core/inter

fingermillet_future_SSP126_IPSL-CM6A-LR_Africa.csv
yams_future_SSP126_MRI-ESM2-0_Africa.csv
groundnut_future_SSP370_IPSL-CM6A-LR_Africa.csv
pigeonpea_future_SSP370_MRI-ESM2-0_Africa.csv


/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/IPython/core/inter

fonio_historical_SSP370_GFDL-ESM4_Africa.csv
fingermillet_future_SSP126_GFDL-ESM4_Africa.csv
sweetpotato_historical_SSP370_GFDL-ESM4_Africa.csv
cowpea_future_SSP126_IPSL-CM6A-LR_Africa.csv
maize_future_SSP126_GFDL-ESM4_Africa.csv


...

/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/IPython/core/inter

groundnut_future_SSP126_GFDL-ESM4_Africa.csv
okra_future_SSP370_GFDL-ESM4_Africa.csv


/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


tef_future_SSP370_GFDL-ESM4_Africa.csv
okra_historical_SSP370_IPSL-CM6A-LR_Africa.csv


/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [53]:
# correcting column names removing the reference to a simulation in the historical column names
for c in grid.columns:
    if "historical" in c:
        grid[c.replace("_ssp370", "")] = grid[c]
        grid = grid.drop(columns=c)


/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: PerformanceWarning: DataFrame is highly fragmented.  T

In [54]:
grid

geometry  yield_taro_future_ssp370_mpi   
0      POINT (19.25000 -34.25000)                           NaN  \
1      POINT (19.75000 -34.25000)                           NaN   
2      POINT (20.25000 -34.25000)                           NaN   
3      POINT (20.75000 -34.25000)                           NaN   
4      POINT (21.25000 -34.25000)                           NaN   
...                           ...                           ...   
10230    POINT (9.25000 36.75000)                           NaN   
10231    POINT (9.75000 36.75000)                           NaN   
10232   POINT (10.25000 36.75000)                           NaN   
10233   POINT (10.75000 36.75000)                           NaN   
10234    POINT (9.75000 37.25000)                           NaN   

       biomass_taro_future_ssp370_mpi  duration_taro_future_ssp370_mpi   
0                                 NaN                              NaN  \
1                                 NaN                              NaN   
2                                 NaN                              NaN   
3                                 NaN                              NaN   
4                                 NaN                              NaN   
...                               ...                              ...   
10230                             NaN                              NaN   
10231                             NaN                              NaN   
10232                             NaN                              NaN   
10233                             NaN                              NaN   
10234                             NaN                              NaN   

       yield_fonio_future_ssp126_mri  biomass_fonio_future_ssp126_mri   
0                                NaN                              NaN  \
1                                NaN                              NaN   
2                                NaN                              NaN   
3                                NaN                              NaN   
4                                NaN                              NaN   
...                              ...                              ...   
10230                            NaN                              NaN   
10231                            NaN                              NaN   
10232                            NaN                              NaN   
10233                            NaN                              NaN   
10234                            NaN                              NaN   

       duration_fonio_future_ssp126_mri   
0                                   NaN  \
1                                   NaN   
2                                   NaN   
3                                   NaN   
4                                   NaN   
...                                 ...   
10230                               NaN   
10231                               NaN   
10232                               NaN   
10233                               NaN   
10234                               NaN   

       yield_fingermillet_future_ssp126_ipsl   
0                                        NaN  \
1                                        NaN   
2                                        NaN   
3                                        NaN   
4                                        NaN   
...                                      ...   
10230                                    NaN   
10231                                    NaN   
10232                                    NaN   
10233                                    NaN   
10234                                    NaN   

       biomass_fingermillet_future_ssp126_ipsl   
0                                          NaN  \
1                                          NaN   
2                                          NaN   
3                                          NaN   
4                                          NaN   
...                                        ...   
10230     

In [55]:
geojson_no_nans(grid, "./unsynced-data/summarized-crop-yield-data/crop-yields-all-models.geojson")
grid.to_csv("./unsynced-data/summarized-crop-yield-data/crop-yields-all-models.csv", index=False) 

In [56]:
grid_reduced = reduce_models(grid)

/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: PerformanceWarning: DataFrame is highly fragmented.  T

In [57]:
grid_reduced

,geometry,duration_sorghum_historical,yield_sweetpotato_future_ssp126,duration_sesame_future_ssp126,duration_grasspea_future_ssp370,yield_fingermillet_future_ssp370,biomass_okra_historical,biomass_sesame_future_ssp126,duration_pigeonpea_future_ssp370,duration_bambaragroundnut_future_ssp370,duration_maize_future_ssp126,biomass_cassava_historical,yield_fingermillet_historical,yield_sesame_future_ssp126,duration_fingermillet_future_ssp370,yield_maize_future_ssp370,yield_josephscoat_future_ssp126,biomass_taro_future_ssp126,duration_fingermillet_historical,duration_pearlmillet_future_ssp370,biomass_cowpea_future_ssp370,duration_tomato_future_ssp126,yield_sorghum_future_ssp370,biomass_fingermillet_historical,duration_josephscoat_future_ssp370,duration_tomato_historical,yield_cassava_future_ssp126,biomass_sweetpotato_future_ssp126,duration_maize_future_ssp370,biomass_maize_future_ssp126,yield_tomato_future_ssp370,duration_groundnut_future_ssp126,duration_sorghum_future_ssp370,yield_okra_future_ssp126,biomass_soybean_future_ssp370,yield_tomato_historical,biomass_cassava_future_ssp126,duration_cocoyam_future_ssp370,duration_africaneggplant_future_ssp126,yield_josephscoat_future_ssp370,biomass_groundnut_historical,duration_josephscoat_historical,duration_cocoyam_future_ssp126,biomass_sweetpotato_future_ssp370,yield_sorghum_historical,biomass_sesame_future_ssp370,duration_lablab_historical,duration_pigeonpea_historical,biomass_yams_future_ssp370,duration_taro_future_ssp126,yield_sweetpotato_future_ssp370,duration_sweetpotato_future_ssp370,biomass_yams_historical,yield_cocoyam_future_ssp126,duration_fonio_future_ssp370,biomass_sorghum_historical,duration_sweetpotato_future_ssp126,biomass_lablab_historical,duration_fonio_historical,duration_bambaragroundnut_future_ssp126,duration_pearlmillet_historical,biomass_okra_future_ssp370,duration_fingermillet_future_ssp126,yield_bambaragroundnut_future_ssp370,biomass_josephscoat_future_ssp126,duration_sesame_historical,biomass_cocoyam_future_ssp126,yield_africaneggplant_future_ssp370,duration_cassava_future_ssp370,biomass_maize_future_ssp370,biomass_groundnut_future_ssp370,duration_tomato_future_ssp370,duration_okra_future_ssp370,duration_sorghum_future_ssp126,biomass_maize_historical,yield_cassava_historical,yield_africaneggplant_future_ssp126,yield_africaneggplant_historical,biomass_africaneggplant_historical,biomass_taro_historical,biomass_africaneggplant_future_ssp370,biomass_lablab_future_ssp370,duration_okra_future_ssp126,biomass_soybean_historical,biomass_africaneggplant_future_ssp126,yield_fonio_historical,yield_cocoyam_historical,biomass_soybean_future_ssp126,yield_cowpea_future_ssp370,biomass_fingermillet_future_ssp370,biomass_josephscoat_future_ssp370,duration_taro_historical,duration_josephscoat_future_ssp126,yield_tef_future_ssp370,duration_mungbean_future_ssp370,duration_lablab_future_ssp126,yield_lablab_future_ssp370,duration_soybean_historical,duration_pigeonpea_future_ssp126,duration_bambaragroundnut_historical,yield_groundnut_future_ssp126,yield_sorghum_future_ssp126,yield_yams_future_ssp126,yield_grasspea_future_ssp370,biomass_yams_future_ssp126,duration_yams_historical,yield_yams_future_ssp370,biomass_josephscoat_historical,duration_pearlmillet_future_ssp126,duration_mungbean_historical,biomass_groundnut_future_ssp126,yield_soybean_future_ssp370,biomass_bambaragroundnut_future_ssp370,duration_grasspea_future_ssp126,duration_maize_historical,yield_pearlmillet_historical,biomass_tef_historical,yield_cassava_future_ssp370,yield_cowpea_future_ssp126,yield_lablab_future_ssp126,biomass_sorghum_future_ssp126,biomass_sesame_historical,biomass_okra_future_ssp126,duration_sweetpotato_historical,biomass_tomato_historical,biomass_pearlmillet_future_ssp370,biomass_fingermillet_future_ssp126,yield_taro_future_ssp126,yield_maize_historical,biomass_sorghum_future_ssp370,biomass_cocoyam_future_ssp370,biomass_cocoyam_historical,biomass_sweetpotato_historical,duration_africaneggplant_future_ssp370

In [58]:
geojson_no_nans(grid_reduced, "./synced-data/crop-yields-mean-models.geojson")
grid_reduced.to_csv("./synced-data/crop-yields-mean-models.csv", index=False)
grid_reduced.to_csv("../vacs-map-app/public/data-raw/crop-yields-mean-models.csv", index=False)

In [59]:
geojson_no_nans(grid_reduced, "./unsynced-data/summarized-crop-yield-data/crop-yields-mean-models.geojson")
grid_reduced.to_csv("./unsynced-data/summarized-crop-yield-data/crop-yields-mean-models.csv", index=False)

In [60]:
quantiles = generate_quantiles(grid_reduced)
with open('./synced-data/crop-yield-data-quantiles.json', 'w') as fp:
    json.dump(quantiles, fp)
quantiles

{'duration_sorghum_historical': {'min': 21.034,
  'quantile_1': 30.26701,
  'quantile_2': 36.407000000000004,
  'quantile_10': 48.1604,
  'quantile_20': 56.9602,
  'quantile_30': 63.1572,
  'quantile_40': 67.825,
  'quantile_50': 71.7295,
  'quantile_60': 75.6742,
  'quantile_70': 80.47229999999999,
  'quantile_80': 86.742,
  'quantile_90': 96.3439,
  'quantile_98': 125.53711999999994,
  'quantile_99': 146.29701000000037,
  'max': 251.083},
 'yield_sweetpotato_future_ssp126': {'min': 5.483,
  'quantile_1': 323.31807,
  'quantile_2': 366.17304,
  'quantile_10': 556.4054,
  'quantile_20': 998.5464000000001,
  'quantile_30': 1743.2870999999998,
  'quantile_40': 2731.9296,
  'quantile_50': 3883.7200000000003,
  'quantile_60': 4883.048,
  'quantile_70': 5454.087099999999,
  'quantile_80': 6090.885800000001,
  'quantile_90': 7459.143300000001,
  'quantile_98': 9133.64778,
  'quantile_99': 9972.715359999993,
  'max': 11560.586},
 'duration_sesame_future_ssp126': {'min': 45.675,
  'quantile_1'

ratio_quantiles = generate_ratio_quantiles(grid_reduced)
with open('./synced-data/crop-yield-data-ratio-quantiles.json', 'w') as fp:
    json.dump(ratio_quantiles, fp)
ratio_quantiles